## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

**1.** Загрузите датасет. Описание датасета можно посмотреть http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/ 

**2.** Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

**3.** Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [1]:
import pandas as pd

In [2]:
SMS = pd.read_csv('SMSSpamCollection.txt', sep='\t', names=['Label', 'Text'])
SMS.head()

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
mapping = {'ham':0, 'spam':1}
SMS['NLabel'] = list(map(lambda x: mapping[x], SMS.Label))
SMS.head()

,Label,Text,NLabel
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


**4.** Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
CV = CountVectorizer()
X = CV.fit_transform(SMS.Text)
print(type(X), X.shape)

<class 'scipy.sparse.csr.csr_matrix'> (5572, 8713)


**5.** Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.model_selection.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. 

In [6]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import cross_val_score as CVS
from sklearn.metrics import f1_score as F1
from sklearn.metrics import make_scorer

In [7]:
clf = LR(random_state=2)
buf = CVS(clf, X, SMS.NLabel, cv=10, scoring=make_scorer(F1)).mean()
print(buf)

0.932640298361


**6.** А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

In [8]:
Messages = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time",
            "Have you visited the last lecture on physics?",
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
            "Only 99$"]

In [9]:
clf.fit(X, SMS.NLabel)
print(clf.predict(CV.transform(Messages)))

[1 1 0 0 0]


**7.** Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [10]:
for CV in [CountVectorizer(ngram_range=(2,2)),
          CountVectorizer(ngram_range=(3,3)),
          CountVectorizer(ngram_range=(1,3))]:
    X = CV.fit_transform(SMS.Text)
    print(CVS(clf, X, SMS.NLabel, cv=10, scoring=make_scorer(F1)).mean())

0.822422066419
0.725016155547
0.925138255865


**8.** Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [11]:
from sklearn.naive_bayes import MultinomialNB as MNB

In [12]:
clf = MNB()
for CV in [CountVectorizer(ngram_range=(2,2)),
          CountVectorizer(ngram_range=(3,3)),
          CountVectorizer(ngram_range=(1,3))]:
    X = CV.fit_transform(SMS.Text)
    print(CVS(clf, X, SMS.NLabel, cv=10, scoring=make_scorer(F1)).mean())

0.645501517799
0.378719485246
0.888485965606


**9.** Попробуйте использовать в логистической регрессии в качестве признаков Tfidf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
TV = TfidfVectorizer()
X = TV.fit_transform(SMS.Text)
clf = LR(random_state=2)
buf1 = CVS(clf, X, SMS.NLabel, cv=10, scoring=make_scorer(F1)).mean()
print('CountVectorizer: ', buf, '; TfIdfVectorizer: ', buf1)

CountVectorizer:  0.932640298361 ; TfIdfVectorizer:  0.852859955417
